Decorator 101 Examples

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running a target()')

# The above is functionally the same as:
# def target():
#     print('running target()')
# target = deco(target)

target()
print(target)

running inner()
None
<function deco.<locals>.inner at 0x757eecd23ba0>


When Python Executes a Decorator Examples

In [5]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1')

@register
def f2():
    print('running f2')

def f3():
    print('running f3')

def main():
    print('running main()')
    print('registry -> ', registry)
    f1()
    f2()
    f3()

main()

running register(<function f1 at 0x757eecd90f40>)
running register(<function f2 at 0x757eecd93e20>)
running main()
registry ->  [<function f1 at 0x757eecd90f40>, <function f2 at 0x757eecd93e20>]
running f1
running f2
running f3


Variable Scope Rules Examples

In [7]:
def fu1(a):
    print(a)
    print(b) 

# fu1(3) # Returns NameError because b was never defined

b = 6
fu1(3)

def fu2(a):
    print(a)
    # Returns UnboundLocalError, because b is now considered local, and called before assignment
    # print(b) 
    # b = 9

def fu3(a):
    global b
    print(a)
    print(b)
    b = 9

3
6


Closures Examples

In [10]:
from typing import Any


class Averager():
    
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

def make_averager():
    series = []
    # The closure for averager extends its scope to include the binding for free variables.
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

avg1 = make_averager()
print(avg1(10))
print(avg1(11))
print(avg1(12))

10.0
10.5
11.0
10.0
10.5
11.0


Standard Library Decorators Examples

In [12]:
from clockdeco import clock

@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
print(fibonacci(6))

[0.00000031s] fibonacci(0) -> 0
[0.00000043s] fibonacci(1) -> 1
[0.00006875s] fibonacci(2) -> 1
[0.00000017s] fibonacci(1) -> 1
[0.00000016s] fibonacci(0) -> 0
[0.00000013s] fibonacci(1) -> 1
[0.00001503s] fibonacci(2) -> 1
[0.00003191s] fibonacci(3) -> 2
[0.00011719s] fibonacci(4) -> 3
[0.00000011s] fibonacci(1) -> 1
[0.00000010s] fibonacci(0) -> 0
[0.00000013s] fibonacci(1) -> 1
[0.00001343s] fibonacci(2) -> 1
[0.00002646s] fibonacci(3) -> 2
[0.00000011s] fibonacci(0) -> 0
[0.00000013s] fibonacci(1) -> 1
[0.00001342s] fibonacci(2) -> 1
[0.00000010s] fibonacci(1) -> 1
[0.00000012s] fibonacci(0) -> 0
[0.00000013s] fibonacci(1) -> 1
[0.00001379s] fibonacci(2) -> 1
[0.00002619s] fibonacci(3) -> 2
[0.00005149s] fibonacci(4) -> 3
[0.00009030s] fibonacci(5) -> 5
[0.00022215s] fibonacci(6) -> 8
8


In [15]:
import functools

from clockdeco import clock

@functools.cache
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(6))

[0.00000022s] fibonacci(0) -> 0
[0.00000044s] fibonacci(1) -> 1
[0.00005933s] fibonacci(2) -> 1
[0.00000061s] fibonacci(3) -> 2
[0.00007663s] fibonacci(4) -> 3
[0.00000050s] fibonacci(5) -> 5
[0.00009245s] fibonacci(6) -> 8
8


In [20]:
import html
import fractions
import decimal

def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

print(htmlize({1, 2, 3}))
print(abs)
print(htmlize('Person and co \n a game'))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))
print(htmlize(True))
print(htmlize(fractions.Fraction(2, 3)))
print(htmlize(2/3))
print(htmlize(decimal.Decimal('0.02380952')))

# Note how this does not change the html tags based on the html object type.

<pre>{1, 2, 3}</pre>
<built-in function abs>
<pre>&#x27;Person and co \n a game&#x27;</pre>
<pre>42</pre>
<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>
<pre>True</pre>
<pre>Fraction(2, 3)</pre>
<pre>0.6666666666666666</pre>
<pre>Decimal(&#x27;0.02380952&#x27;)</pre>


In [8]:
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

@singledispatch
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register
def _(text: str) -> str:
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

@htmlize.register
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

@htmlize.register(decimal.Decimal)
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'

print(htmlize({1, 2, 3}))
print(htmlize(abs))
print(htmlize('Person and co \n- a game'))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))
print(htmlize(True))
print(htmlize(fractions.Fraction(2, 3)))
print(htmlize(2/3))
print(htmlize(decimal.Decimal('0.02380952')))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<p>Person and co <br/>
- a game</p>
<pre>42 (0x2a)</pre>
<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>
<pre>True</pre>
<pre>2/3</pre>
<pre>0.6666666666666666 (2/3)</pre>
<pre>0.02380952 (1/42)</pre>


Parameterized Decorators Examples

In [12]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def fv1():
    print('running f1()')

print('running main()')
print('running -> ', registry)
print(fv1) 

running register(<function fv1 at 0x770dbeb9b740>)
running main()
running ->  [<function fv1 at 0x770dbeb9b740>]
<function fv1 at 0x770dbeb9b740>


In [16]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register' f'(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=False)
def fc1():
    print('running fc1()')
    
@register()
def fc2():
    print('running fc2()')

def fc3():
    print('running fc3()')

print(registry)
print(register()(fc3))
print(registry)
print(register(active=False)(fc2))
print(registry)

running register(active=False)->decorate(<function fc1 at 0x770dbd739e40>)
running register(active=True)->decorate(<function fc2 at 0x770dbd739d00>)
{<function fc2 at 0x770dbd739d00>}
running register(active=True)->decorate(<function fc3 at 0x770dbd739c60>)
<function fc3 at 0x770dbd739c60>
{<function fc2 at 0x770dbd739d00>, <function fc3 at 0x770dbd739c60>}
running register(active=False)->decorate(<function fc2 at 0x770dbd739d00>)
<function fc2 at 0x770dbd739d00>
{<function fc3 at 0x770dbd739c60>}


In [20]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

if __name__ == '__main__':
    @clock()
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)

[0.12318084s] snooze(0.123) -> None
[0.12320078s] snooze(0.123) -> None
[0.12307316s] snooze(0.123) -> None


In [23]:
import time
from clockdeco_param import clock

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12316481500056398s
snooze: 0.12312400900009379s
snooze: 0.12309593499958282s


In [25]:
import time
from clockdeco_param import clock

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s


In [26]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class clock:
    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt
    
    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked